In [9]:
import pandas as pd
import requests
from time import sleep

In [10]:
url  ='http://dadosabertos.rio.rj.gov.br/apiTransporte/apresentacao/rest/index.cfm/obterTodasPosicoes'

In [13]:
def get_data():
    try:
        data = requests.get(url).json()
        df = pd.DataFrame.from_records(data['DATA'], columns=data['COLUMNS'])
        df['DATAHORA'].apply(lambda x: pd.datetime.strptime(x, "%m-%d-%Y %H:%M:%S"))
        return df
    except:
        return pd.DataFrame()

In [15]:
get_data()

""


In [16]:
try:
    all_data = pd.read_csv('all_data.csv')
except:
    all_data = pd.DataFrame()


In [17]:
all_data.tail()

,DATAHORA,ORDEM,LINHA,LATITUDE,LONGITUDE,VELOCIDADE
72332,2020-04-05 15:44:18,A71557,409.0,-22.96372,-43.21699,0.0
72333,2020-04-05 15:44:19,A71615,100.0,-22.98479,-43.19864,0.0
72334,2020-04-05 15:44:19,B82626,678.0,-22.88632,-43.25134,16.0
72335,2020-04-05 15:44:20,A71523,409.0,-22.94313,-43.17628,42.0
72336,2020-04-05 15:44:20,B71074,455.0,-22.90287,-43.18370,14.0


In [7]:
while True:
    try:
        all_data = all_data.append(get_data(), ignore_index=True)
        all_data = all_data.drop_duplicates(keep='first')
        all_data.to_csv('all_data.csv', index=False)
        print(all_data.shape)
        sleep(180)
    

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))